# Semantic Kernel OpenAI Assistant Agent Code Interpreter

## Prepare File Paths of our files to be added to agent

In [1]:
import os

# Let's form the file paths that we will later pass to the assistant
csv_file_path_1 = os.path.join("../Data/population/","PopulationByAdmin1.csv",)

csv_file_path_2 = os.path.join("../Data/population/","PopulationByCountry.csv",)

## Step 1-2: Create an Agent and Thread

In [2]:
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.file_reference_content import FileReferenceContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

# Step 1: Create an assistant agent
agent = await AzureAssistantAgent.create(
        kernel=Kernel(),
        service_id="agent",
        name="SK_OpenAI_Assistant_Agent_Code_Interpreter",
        instructions="""
                Analyze the available data to provide an answer to the user's question.
                Always format response using markdown.
                Always include a numerical index that starts at 1 for any lists or tables.
                Always sort lists in ascending order.
                """,
        enable_code_interpreter=True,
        code_interpreter_filenames=[csv_file_path_1, csv_file_path_2],
)

# Step 2: Create a thread
thread_id = await agent.create_thread()

## Helper Methods
This will download any files created by the agent

In [3]:
async def download_file_content(agent, file_id: str):
    try:
        # Fetch the content of the file using the provided method
        response_content = await agent.client.files.content(file_id)

        # Get the current working directory of the file
        current_directory = "../Data/skagents-output/"

        # Define the path to save the image in the current directory
        file_path = os.path.join(
            current_directory,  # Use the current directory of the file
            f"{file_id}.png",  # You can modify this to use the actual filename with proper extension
        )

        # Save content to a file asynchronously
        with open(file_path, "wb") as file:
            file.write(response_content.content)

        print(f"File saved to: {file_path}")
    except Exception as e:
        print(f"An error occurred while downloading file {file_id}: {str(e)}")


async def download_response_image(agent, file_ids: list[str]):
    if file_ids:
        # Iterate over file_ids and download each one
        for file_id in file_ids:
            await download_file_content(agent, file_id)

## Step 3-5: Helper Function
3. Add a message to the thread
4. Run the Assistant
5. Display the Assistant's Response

In [4]:
async def run_agent(user_question):
    # File IDs to store the references to the files
    file_ids: list[str] = []

    # STEP 3: Add a user question to the thread
    await agent.add_chat_message(
        thread_id=thread_id, 
        message=ChatMessageContent(role=AuthorRole.USER, content=user_question)
    )

    # STEP 4: Invoke the agent to get a response
    async for response in agent.invoke(thread_id=thread_id):
        # check if the response contains any code
        if response.metadata.get("code"):
            print("\n" + "Code from Agent:")
            #STEP 5: Print the Assistant response
            print(f"{response.content}", end="", flush=True)        
        else:
            #STEP 5: Print the Assistant response
            print(f"{response.content}", end="", flush=True)
                    
        print("\n**************")
        #Check if the response contains any file references
        file_ids.extend([
            item.file_id for item in response.items if isinstance(item, FileReferenceContent)
        ])

    # Download the file content
    await download_response_image(agent, file_ids)
    file_ids.clear()


In [5]:
user_question = "What is the population of the country with the highest population?"
await run_agent(user_question)


Code from Agent:
import pandas as pd

# Load the data from the uploaded files
file_1 = '/mnt/data/assistant-JjnuKunz2Qn5eZT8wuzYtG'
file_2 = '/mnt/data/assistant-4UHpGiZJ5QXYCKJtD9HQ2H'

# Display the content of the files
data_1 = pd.read_excel(file_1)
data_2 = pd.read_excel(file_2)

data_1.head(), data_2.head()
**************

Code from Agent:
# Try loading the data again but specifying the excel engine as 'openpyxl' in case it's an .xlsx
data_1 = pd.read_excel(file_1, engine='openpyxl')
data_2 = pd.read_excel(file_2, engine='openpyxl')

data_1.head(), data_2.head()
**************
It appears that the files are not recognized as Excel (.xlsx) files. They might be in a different format. Let's attempt to load the files as CSVs or other common data formats and inspect their content.
**************

Code from Agent:
# Try loading the data as CSV files
data_1 = pd.read_csv(file_1)
data_2 = pd.read_csv(file_2)

data_1.head(), data_2.head()
**************
The first file (`data_1`) contains p

## Appending Messages to the Thread

In [6]:
user_question = "What is in third place?"
await run_agent(user_question)


Code from Agent:
# Sort data by population in descending order and get the third highest population country
sorted_data = data_1.sort_values(by='Population', ascending=False)
third_place_country = sorted_data.iloc[2]

third_place_country[['Country_Region', 'Population']]
**************
The country in third place in terms of population is the **United States** with a population of **329,466,283**.
**************


In [7]:
user_question = "Give me a column chart of the top 10 countries by population. Add values at the top of each column"
await run_agent(user_question)


Code from Agent:
import matplotlib.pyplot as plt

# Get the top 10 countries by population
top_10_countries = sorted_data.head(10)

# Create a bar plot
plt.figure(figsize=(14, 8))
bars = plt.bar(top_10_countries['Country_Region'], top_10_countries['Population'], color='skyblue')

# Add values on top of each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2 - 0.2, yval + 10000000, int(yval), va='bottom') 

plt.xlabel("Country")
plt.ylabel("Population")
plt.title("Top 10 Countries by Population")
plt.xticks(rotation=45)
plt.show()
**************
Here is a column chart displaying the top 10 countries by population, with the population values displayed at the top of each column.
**************
File saved to: ../Data/skagents-output/assistant-EAifEhQZZTYbdKa3eJJuCf.png


In [ ]:
user_question = "Provide a column chart for countries whose names start with the same letter and sort the x axis by highest count to lowest (include all countries). Add values on top of each column."
await run_agent(user_question)


Code from Agent:
# Create a bar plot with the specifications mentioned and save it as a JPG
plt.figure(figsize=(14, 8))
bars = plt.bar(letter_counts.index, letter_counts.values, color='lightcoral')

# Add values on top of each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2 - 0.1, yval + 0.2, int(yval), va='bottom') 

plt.xlabel("First Letter of Country Name")
plt.ylabel("Number of Countries")
plt.title("Number of Countries by First Letter of Name")
plt.xticks(rotation=0)

# Save the plot as a JPG
plt.savefig('/mnt/data/countries_by_first_letter.jpg', format='jpg')
plt.show()
**************
The column chart has been saved as a JPG file. You can download it using the link below:

[Download the chart](sandbox:/mnt/data/countries_by_first_letter.jpg)
**************
File saved to: ../Data/skagents-output/assistant-XiyxAvdeGFgQJDNKLpww2U.png


## Deleting Files, Thread, Agent

In [11]:
if agent is not None:
    [await agent.delete_file(file_id) for file_id in agent.code_interpreter_file_ids]
    await agent.delete_thread(thread_id)
    await agent.delete()